In [ ]:
# Importing Local Functions
import sys
sys.path.append("../functions")
sys.path.append("../inputs")

from gcs_ingestion import *
from shared import *

In [ ]:
# Reading Script Input Variables
with open("../inputs/gcs_ingestion_settings.json") as f:
    gcs_ingestion_settings = json.load(f)

start_date, end_date = script_date_selection(gcs_ingestion_settings)
script_setting       = gcs_ingestion_settings.get("script_setting")
headers              = gcs_ingestion_settings.get("request_headers")
project_name         = gcs_ingestion_settings.get("project_id")
bucket_name          = gcs_ingestion_settings.get("bucket_name")

# Initialise Logger Object
logger = initialise_cloud_logger(project_name)
logger.info(f"Project: {project_name} | Bucket: {bucket_name} | Script Setting: {script_setting} | Ingestion Dates: {start_date} - {end_date} ")

In [ ]:
# Getting current leaderboard data of top chess players
logger.info('Requesting the latest leaderboards')
leaderboards_url = f'https://api.chess.com/pub/leaderboards'
leaderboards_response = exponential_backoff_request(leaderboards_url, headers, logger)
gcs_leaderboard_endpoint = f"leaderboards/{datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}"
upload_json_to_gcs_bucket(bucket_name, gcs_leaderboard_endpoint, leaderboards_response, logger)

In [ ]:
# Determine list of requests for players and specified period
year_month_list = generate_year_month_list(start_date, end_date)
top_player_list =  get_top_player_list(leaderboards_response, logger)
remaining_request_list = get_request_permutations(bucket_name, top_player_list, year_month_list, logger)

# Iterating through each request permutation and uploading to GCS
request_all_games_and_upload_to_gcs(bucket_name, top_player_list, year_month_list, headers, logger)